In [18]:
!pip install emoji

You are using pip version 19.0.3, however version 19.2.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [19]:
!pip install pandas

You are using pip version 19.0.3, however version 19.2.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [20]:
import numpy as np
import pandas as pd

In [21]:
train = pd.read_csv('dataset/train_emoji.csv',header=None)
test = pd.read_csv('dataset/test_emoji.csv',header=None)

In [22]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [23]:
import emoji as emoji

In [24]:
#emoji.EMOJI_ALIAS_UNICODE

In [25]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [26]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [27]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [28]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [29]:
from keras.utils import to_categorical

ModuleNotFoundError: No module named 'keras'

In [ ]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

In [ ]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [ ]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [ ]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

In [ ]:
print(emb_XT.shape)
print(emb_Xt.shape)


In [ ]:
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

In [ ]:
model.evaluate(emb_Xt,Yt)

In [ ]:
pred = model.predict_classes(emb_Xt)

In [ ]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

In [ ]:
with open("model.json", "w") as file:
    file.write(model.to_json())

In [ ]:
from keras.models import model_from_json

In [ ]:
with open("model.json", "r") as file:
    model = model_from_json(file.read())

In [ ]:
test_str = "Hello how are you"
X = pd.Series([test_str])

In [ ]:
emb_X = getOutputEmbeddings(X)

In [ ]:
p = model.predict_classes(emb_X)

In [ ]:
print(' '.join(X[0]))
print(emoji.emojize(emoji_dictionary[str(p[0])]))

In [30]:
from emoji import emojize
import numpy as np
import pandas as pd
from keras.models import model_from_json

print("-- Loading Model File")
model_json_file = 'model.json'
with open(model_json_file, "r") as file:
    model = model_from_json(file.read())

print("-- Loading Embeddings File")
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        embeddings[word] = coeffs

def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

def predict(input_str):
    emoji_dictionary = {
        "0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
        "1": ":baseball:",
        "2": ":beaming_face_with_smiling_eyes:",
        "3": ":downcast_face_with_sweat:",
        "4": ":fork_and_knife:",
    }

    X = pd.Series([test_str])
    emb_X = getOutputEmbeddings(X)
    p = model.predict_classes(emb_X)

    return emoji_dictionary[str(p[0])]

if __name__ == "__main__":
    test_str = "i love it"
    output = predict(test_str)
    print(emojize(output))


ModuleNotFoundError: No module named 'keras'